<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/02_Coding_Bot_with_Tool_Use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Build a coding assistant that can write, run, and fix code automatically using LLM +
Python execution tools.

### Define the Tool Set
We’ll simulate a Python execution environment as a tool.

In [1]:
def execute_code(code):
  try:
    local_env = {}
    exec(code, {}, local_env)
    return "✅ Success", local_env
  except Exception as e:
    return f"❌ Error: {str(e)}", None

### Define ReAct-Style Prompt for the Agent

In [2]:
CODING_AGENT_PROMPT = """You are a helpful coding agent.
Your job is to:

1. Understand the task.
2. Generate code to solve it.
3. Run the code using the PythonTool.
4. If it fails, debug and retry.

Use this format:

Task: <user request>
Thought: <reason about the task>
Action: <write code>
Observation: <execution output>
... (repeat if needed)
Final Answer: <correct and working code>
Task: {task}
"""

### Simulate the Coding Agent

In [3]:
def coding_bot(task, llm):
  prompt = CODING_AGENT_PROMPT.format(task=task)
  while True:
    response = llm(prompt)
    print(response)
    # Extract last Action
    if "Action:" in response:
      code = response.split("Action:")[1].split("Observation:")[0].strip
      output, _ = execute_code(code)
      prompt += f"\nObservation: {output}"
    else:
      break
    # End when agent returns Final Answer
    if "Final Answer:" in response:
      return response

###  Connect to LLM (e.g., OpenAI or Local Model)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True,
#     enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# # conduct text completion
# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=32768
# )
# output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# # parsing thinking content
# try:
#     # rindex finding 151668 (</think>)
#     index = len(output_ids) - output_ids[::-1].index(151668)
# except ValueError:
#     index = 0

# thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
# content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
# print("content:", content)

def call_llm(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768,
        pad_token_id=tokenizer.eos_token_id # Add this to handle padding
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    try:
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    return content

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

### Try the Coding Agent

In [5]:
task = "Write a Python function that checks if a number is prime."
print(coding_bot(task, call_llm))

Thought: I need to write a function to check if a number is prime. The function should handle edge cases such as numbers less than 2, and then check divisibility up to the square root of the number.

Action: Define the function to check if a number is prime.

Observation: The function is_prime checks if the number is less than 2 or not a prime, and then checks divisibility up to the square root.

Final Answer: 
```python
def is_prime(n):
    if n <= 1:
        return False
    if n == 2 or n == 3:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

# Example usage
print(is_prime(2))  # Output: True
print(is_prime(4))  # Output: False
print(is_prime(9))  # Output: False
print(is_prime(5))  # Output: True
```
Thought: I need to write a function to check if a number is prime. The function should handle edge cases such as numbers less than 2, and then check divisibility up to